In [70]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re
import time # we'll use this for timing how long things take...
import urllib3


# SCRAPE THE SCANLATOR GROUPS

In [71]:
page = "https://www.mangaupdates.com/groups.html?id=8904"
req = requests.get(page)
d_soup = bs(req.content)
OtherInfo = d_soup.find_all("div", "p-1 col-6 text")

In [72]:
def GroupID(page):
    groupID = re.findall(r'id=([0-9]+)',page)[0]
    return groupID

In [73]:
def GroupName(d_soup):
    GroupName = d_soup.find_all("div", "p-1 col-6 specialtext")
    return GroupName[0].text

In [74]:
def ReleaseFrequency(OtherInfo):
    ReleaseFrequency = OtherInfo[8].text
    return ReleaseFrequency

In [75]:
def NumberofRelease(OtherInfo):
    NumberofRelease = OtherInfo[10].text
    return NumberofRelease

In [76]:
def Active_or_Not(OtherInfo):
    Active_or_Not = OtherInfo[12].text
    return Active_or_Not

In [77]:
def Total_Series(OtherInfo):
    Total_Series = OtherInfo[14].text
    return Total_Series

In [78]:
def Genres_Sort(OtherInfo):
    Genres = OtherInfo[16].text 
    All_Genres = ['Action','Adult','Adventure','Comedy','Doujinshi','Drama','Ecchi','Fantasy','Gender Bender','Harem','Hentai','Historical','Horror','Josei','Lolicon','Martial Arts','Mature','Mecha','Mystery','Psychological','Romance','School Life','Sci-fi','Seinen','Shotacon','Shoujo','Shoujo Ai','Shounen', 'Shounen Ai','Slice of Life','Smut','Sports','Supernatural','Tragedy','Yaoi','Yuri']
    dict_genres = {}
    genre_name_num = re.findall(r"(.*?)[\s]*[(](.*?)[)][,\s]*",Genres)
    for genre in All_Genres:
        dict_genres[genre] = 0
        for item in genre_name_num:
            if item[0]==genre:
                dict_genres[genre] = item[1]
    return dict_genres


In [79]:
def get_last_updated_day(d_soup):
    if d_soup.find_all("div","col-2 text") == []:
        last_updated_day = "no date"
    else:
        last_updated_day = d_soup.find_all("div", "col-2 text")[0].text
    return last_updated_day

In [80]:
def founding_day_new(page, d_soup):
    #IF THE GROUP ONLY HAVE ONE RELEASE PAGE
    if d_soup.find_all("span","d-none") == []:
        if d_soup.find_all("div","col-2 text") == []:
            day = "no date"
        else:

            #IN CASE THERE IS ONLY ONE RELEASE PAGE
            #In case there is only one date in release page
            day_raw = d_soup.find_all("div","col-2 text")
            day_list = [i.text for i in day_raw if i.text != "Unknown"]
            if day_list == []:
                day = "no date"
            else:
                day = day_list[len(day_list)-1]
                     
    else:
        all_day = []
    #FIRST, GET THE NUMBER OF THE GROUP'S RELEASE PAGES
        total_page_unclean = d_soup.find_all("span","d-none")[0].text.strip()
        page_series_number = re.findall(r"[0-9]+",total_page_unclean)[0]
        #GO TO THE LAST AND SECOND LAST RELEASE PAGES
        for j in range(len(page)): 
            if page[j] == "?":
                insert = j
        for i in range(int(page_series_number)-1, int(page_series_number)+1):
            turn_page = page[:(insert+1)] + "page={}&".format(i) + page[(insert+1):]

        #TAKE THE FOUNDING DAY (BE CAREFUL SINCE SOME OF DAYS ARE UNKNOWN)
            req_founding_day_page_link = myGet(turn_page)
            soup_founding_day_page_content = bs(req_founding_day_page_link.content)
            founding_day_init = soup_founding_day_page_content.find_all("div","col-2 text")
            all_day = all_day + [i.text for i in founding_day_init if i.text != "Unknown"]
        if all_day == []:
            day = "no date"
        else: 
            day = all_day[len(all_day)-1]
    return day

In [81]:
group_dict = {"Name":GroupName(d_soup),"Active or not":Active_or_Not(OtherInfo),"Release Frequency":ReleaseFrequency(OtherInfo),"Number of Release":NumberofRelease(OtherInfo),"Total Series":Total_Series(OtherInfo),"First Update":founding_day_new(page, d_soup),"Lastest Update":get_last_updated_day(d_soup)}

In [222]:
group_dict.update(Genres_Sort(OtherInfo))

In [223]:
group_dict

{'Name': 'Lemonade Scans',
 'Active or not': 'Yes',
 'Release Frequency': 'Often. About every 7 to 8 day(s)',
 'Number of Release': '410',
 'Total Series': '74',
 'First Update': '04/03/17',
 'Lastest Update': '08/04/20',
 'Action': 0,
 'Adult': 0,
 'Adventure': 0,
 'Comedy': '18',
 'Doujinshi': 0,
 'Drama': '18',
 'Ecchi': 0,
 'Fantasy': 0,
 'Gender Bender': '1',
 'Harem': '2',
 'Hentai': 0,
 'Historical': 0,
 'Horror': 0,
 'Josei': '5',
 'Lolicon': 0,
 'Martial Arts': 0,
 'Mature': 0,
 'Mecha': 0,
 'Mystery': '1',
 'Psychological': '2',
 'Romance': '65',
 'School Life': '45',
 'Sci-fi': 0,
 'Seinen': 0,
 'Shotacon': 0,
 'Shoujo': '54',
 'Shoujo Ai': 0,
 'Shounen': 0,
 'Shounen Ai': '1',
 'Slice of Life': '16',
 'Smut': '1',
 'Sports': '1',
 'Supernatural': '2',
 'Tragedy': 0,
 'Yaoi': '8',
 'Yuri': 0}

In [224]:
group_cell = pd.DataFrame.from_dict(group_dict, orient='index').transpose()


In [82]:
#Final function to create a group cell

def group_cell(page):
    req = myGet(page)
    d_soup = bs(req.content)
    OtherInfo = d_soup.find_all("div", "p-1 col-6 text")
    group_dict = {"Name":GroupName(d_soup),"Active or not":Active_or_Not(OtherInfo),"Release Frequency":ReleaseFrequency(OtherInfo),"Number of Release":NumberofRelease(OtherInfo),"Total Series":Total_Series(OtherInfo),"First Update":founding_day_new(page, d_soup),"Lastest Update":get_last_updated_day(d_soup)}
    group_dict.update(Genres_Sort(OtherInfo))
    group_cell = pd.DataFrame.from_dict(group_dict, orient='index').transpose()
    return group_cell

SCRAPE THE GROUP PAGE

In [147]:
#get the link of each scanlator group
scanlator_link_include = soup_page_scanlator.find_all("a", alt="Group Info")
link_group = scanlator_link_include[0].get('href')
link_group

'https://www.mangaupdates.com/groups.html?id=3143'

TESTING THE FIRST PAGE OF GROUP SCANLATION

In [291]:
page_scanlator = 'https://www.mangaupdates.com/groups.html?page=136&'

In [292]:
# header_new = {"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9","Accept-Encoding": "gzip, deflate, br","Accept-Language": "vi-VN,vi;q=0.9,en-US;q=0.8,en;q=0.7","Connection": "keep-alive", "Host": "www.mangaupdates.com","Sec-Fetch-Dest": "document","Sec-Fetch-Mode": "navigate","Sec-Fetch-Site": "same-origin","Sec-Fetch-User": "?1","Upgrade-Insecure-Requests": "1","User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Mobile Safari/537.36}"}

In [293]:
# headers = header_new
# s.headers.update(header_new)

In [83]:
#Send Http requests with Tor
s = requests.Session()
s.proxies = {}
s.proxies['http'] = 'socks5h://localhost:9150'
s.proxies['https'] = 'socks5h://localhost:9150'
headers = {}
headers['User-agent'] = "HotJava/1.1.2 FCS"
s.cookies.clear()

In [84]:
import random
def myGet(url):
    for i in range(10):
        try:
            pause = random.uniform(1, 3)
            time.sleep(pause)
            s.cookies.clear()
            req = s.get(url, headers=headers, verify=True)
            break
        except:
            continue
    else:
        print("Failed 10 times, skip this" + " " + url)
        req = None
    return req

In [315]:
req_page_scanlator = myGet(page_scanlator)

In [316]:
soup_page_scanlator = bs(req_page_scanlator.content)

In [317]:
scanlator_link_include = soup_page_scanlator.find_all("a", alt="Group Info")

In [318]:
scanlator_link = [i.get('href') for i in scanlator_link_include]

In [319]:
scanlator_link

['https://www.mangaupdates.com/groups.html?id=7222',
 'https://www.mangaupdates.com/groups.html?id=8391',
 'https://www.mangaupdates.com/groups.html?id=9251',
 'https://www.mangaupdates.com/groups.html?id=10669',
 'https://www.mangaupdates.com/groups.html?id=12610',
 'https://www.mangaupdates.com/groups.html?id=4558',
 'https://www.mangaupdates.com/groups.html?id=4675',
 'https://www.mangaupdates.com/groups.html?id=10213',
 'https://www.mangaupdates.com/groups.html?id=9254',
 'https://www.mangaupdates.com/groups.html?id=3298',
 'https://www.mangaupdates.com/groups.html?id=447',
 'https://www.mangaupdates.com/groups.html?id=5500',
 'https://www.mangaupdates.com/groups.html?id=4823',
 'https://www.mangaupdates.com/groups.html?id=2441',
 'https://www.mangaupdates.com/groups.html?id=6300',
 'https://www.mangaupdates.com/groups.html?id=4082',
 'https://www.mangaupdates.com/groups.html?id=9412',
 'https://www.mangaupdates.com/groups.html?id=7260',
 'https://www.mangaupdates.com/groups.html?i

In [261]:
df_group_test = pd.DataFrame()
for i in range(5):
    df_group_test = df_group_test.append(group_cell(scanlator_link[i]))
df_group_test    

,Name,Active or not,Release Frequency,Number of Release,Total Series,First Update,Lastest Update,Action,Adult,Adventure,...,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Smut,Sports,Supernatural,Tragedy,Yaoi,Yuri
0,Hikki no Mori Translations,No,Inactive Group,59,2,03/27/16,08/20/17,2,0,1,...,0,1,0,0,0,0,1,1,0,0
0,Hikoki Translations,No,Inactive Group,30,2,06/25/17,02/16/18,1,0,1,...,0,1,0,0,0,0,0,0,0,0
0,Hikotoshi,Yes,Infrequently,9,1,06/22/19,05/03/20,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,Hillbilly scans,Yes,Infrequently,1,1,08/02/20,08/02/20,1,0,1,...,0,0,0,0,0,0,0,0,0,0
0,Hima Scans,No,Inactive Group,10,2,12/08/11,01/13/12,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
df_group_test.to_csv('scanlator_group_new_60.csv')

In [87]:
def scan_each_page(page_scanlator):
    df_group_test = pd.DataFrame()
    req_page_scanlator = myGet(page_scanlator)
    soup_page_scanlator = bs(req_page_scanlator.content.decode('utf-8', 'ignore'))
    scanlator_link_include = soup_page_scanlator.find_all("a", alt="Group Info")
    scanlator_link = [i.get('href') for i in scanlator_link_include]
    for i in scanlator_link:
        df_group_test = df_group_test.append(group_cell(i))
    return df_group_test

In [267]:
#SCAN EACH PAGE AT ONE TIME

In [86]:
# %%time
import datetime
start_time = datetime.datetime.now()
for i in range(60,61): 
    page_scan_test = 'https://www.mangaupdates.com/groups.html?page={}&'.format(i)
    scan_each_page(page_scan_test).to_csv('scanlator_group_new_{}.csv'.format(i))
    time_process = datetime.datetime.now() - start_time
    print(time_process)
        


0:01:29.318573


In [410]:
request_link18 = requests.get(scanlator_link[9])
soup_link18 = bs(request_link18.content)
founding_day_new_test(scanlator_link[9], soup_link18)

'10/24/19'

In [411]:
scanlator_link[19]

'https://www.mangaupdates.com/groups.html?id=1782'

TESTING TWO PAGES AT ONCE

In [594]:
%%time
df_group_test_2 = pd.DataFrame()
for i in range(51, 53):
    page_number = i
    page_scanlator = 'https://www.mangaupdates.com/groups.html?page={}&'.format(page_number)
    req_page_scanlator = requests.get(page_scanlator)
    soup_page_scanlator = bs(req_page_scanlator.content)
    scanlator_link_all = soup_page_scanlator.find_all("a",alt="Group Info")
    for i in range(len(scanlator_link_all)):
        link_group = scanlator_link_all[i].get('href')
        df_group_test_2 = df_group_test_2.append(group_cell(link_group))
    time.sleep(0.1)
df_group_test_2

Wall time: 1min 12s


,Name,Active or not,Release Frequency,Number of Release,Total Series,First Update,Lastest Update,Action,Adult,Adventure,...,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Smut,Sports,Supernatural,Tragedy,Yaoi,Yuri
0,buzzichan,Yes,Infrequently,1,1,04/02/20,04/02/20,0,0,0,...,0,0,1,0,0,0,0,0,0,0
0,bvnnyscans,Yes,Infrequently,1,1,04/06/20,04/06/20,0,0,0,...,0,0,0,0,0,0,0,0,1,0
0,bxsmanga,No,Inactive Group,20,4,04/10/11,01/12/14,0,0,0,...,0,0,0,1,1,0,1,0,0,0
0,By,No,Inactive Group,5,1,04/01/18,09/30/18,1,0,1,...,0,0,0,0,0,0,0,0,0,0
0,By Myself,Yes,Every day,4,1,05/18/20,05/21/20,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,BYAAAH,No,Inactive Group,60,14,12/25/06,09/06/09,6,0,0,...,0,6,0,3,0,0,3,0,0,0
0,Byakuei-Scans,No,Inactive Group,2,1,07/14/07,07/15/07,1,0,0,...,0,1,0,0,0,0,1,0,0,0
0,byunlisa773,No,Inactive Group,19,1,03/03/17,05/03/17,0,0,0,...,0,0,1,1,0,0,0,0,0,0
0,C,No,Inactive Group,2,1,02/06/06,03/18/06,1,0,0,...,0,1,0,0,0,0,1,0,0,0
0,C-Side Cafe,No,Inactive Group,8,2,07/05/13,06/24/14,1,0,0,...,0,2,0,0,0,0,1,0,0,0


In [333]:
df_group_test_2.to_csv('scanlator_group_bundle_test.csv')

In [59]:
#One group have problem with release date
page_gotnyu = 'https://www.mangaupdates.com/groups.html?id=10935'
req_page_gotnyu = requests.get(page_gotnyu)
soup_page_gotnyu = bs(req_page_gotnyu.content)
date_haveitornot = soup_page_gotnyu.find_all("div","col-2 text")
date_haveitornot

[]

 TEST TO SEE IF THE ABOVE CODES ARE APPLICABLE TO OTHER GROUPS

In [197]:
page_test = 'https://www.mangaupdates.com/groups.html?id=1665'
req_test = requests.get(page_test)
d_soup_test = bs(req_test.content)
d_soup_test

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
<head>
<title>Baka-Updates Manga - ...and other escapades</title>
<meta content="Latest And Newest Manga Release Updates and News." name="description"/>
<meta content="manga, anime, updates, update, releases, release, new, scanlators, scanlator, wiki" name="keywords"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Manick(Code&amp;Html)JYT(Design)" name="author"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="favicon.ico" rel="shortcut icon"/>
<link href="dist/main.26f679a5b9bf274231c0.css" rel="stylesheet" type="text/css"/>
<link href="rss.php" rel="alternate" title="Baka Updates Manga RSS" type="application/rss+xml"/>
<link href="mangaupdates.xml" rel="search" title="MangaUpdates Search" type="application/opensearchdescription+xml"/>
</head>
<body>
<script src="dist/main.35477e88acdffef5682b.js" type="

In [198]:
GroupName_test = d_soup_test.find_all("div", "p-1 col-6 specialtext")
GroupName_test[0].text

'...and other escapades'

In [199]:
OtherInfo_test = d_soup_test.find_all("div", "p-1 col-6 text")
ReleaseFrequency_test = OtherInfo_test[8].text
ReleaseFrequency_test

'Inactive Group'

In [200]:
Genres_test = OtherInfo_test[16].text
Genres_test

'Doujinshi (2), Yaoi (1), Shounen Ai (1)'

In [201]:
d_soup_test.find_all("div", "col-2 text")[0]

<div class="col-2 text">01/28/08</div>

In [202]:
NumberofRelease_test = OtherInfo_test[10].text
NumberofRelease_test

'2'

In [203]:
d_soup_test.find_all("span","d-none")

[]

In [209]:
if d_soup_test.find_all("span","d-none") == []:
    founding_day_index = len(d_soup_test.find_all("div","col-2 text"))-1
    day = d_soup_test.find_all("div","col-2 text")[founding_day_index].text
day

'01/28/08'

In [227]:
founding_day_new_test(d_soup_test, page_test)

NameError: name 'd_soup_test' is not defined

# SCRAPE LICENSED TITLES

In [5]:
def title(soup_series):
    title = soup_series.find_all("span","releasestitle tabletitle")[0].text
    return title

In [6]:
def type_comic(soup_series):
    comictype = soup_series.find_all("div", "sContent")[1].text.strip()
    return comictype

In [7]:
def number_group_scanlate(soup_series):
    groups_scanlate = soup_series.find_all("a", title = "Group Info" )
    total_group = []
    for i in groups_scanlate:
        group = i.text
        if group not in total_group and "Viz" not in group or group not in total_group and "MANGA Plus" not in group or group not in total_group and "Crunchyroll" not in group:
            total_group.append(group)
    return len(total_group)

In [8]:
def AwardorNot(series_id):
#find link of the category words in the network tab of developer tool
    category_list_link = "https://www.mangaupdates.com/ajax/show_categories.php?s="+ series_id[0] + "&type=1"
    request_category = myGet(category_list_link)
    soup_category = bs(request_category.content)
    category_list = soup_category.find_all("li","tag_normal")
    category_list_text = []
    for i in category_list:
        category_list_text.append(i.text)
    if 'Award-Winning Work' in category_list_text or 'Award-Nominated Work' in category_list_text: 
        return True
    else:
        return False
    

In [9]:
def start_scan_date(soup_series, series_id):
    if soup_series.find_all("div","sContent")[5] == "" or soup_series.find_all("div", "sContent")[5].text.strip() == "N/A":
        date = "Pass"
    else:
        release_link = "https://www.mangaupdates.com/releases.html?stype=series&search="+ series_id[0]
        request_release_link = myGet(release_link)
        release_link_soup = bs(request_release_link.content)
        page_number = release_link_soup.find_all("span","d-none")
        if page_number == []:
            all_date_release = release_link_soup.find_all("div", "col-2 text")
            if len(all_date_release) == 1:
                if all_date_release[0].text != "Unknown":
                    date = all_date_release[0].text
                else: 
                    date = "Unknown"
            else:
                for j in range(len(all_date_release)-1,0,-1):
                        if all_date_release[j].text != "Unknown":
                            date = all_date_release[j].text 
                            break 
                        else: 
                            date = "Unknown"
        else:
            page_number_only = re.findall(r'[0-9]+',page_number[0].text)[0]
            page_number_only = int(page_number_only)
            for i in range(page_number_only,page_number_only-2,-1):
                release_start_link = release_link + "&page=" + str(i)
                request_release_start = myGet(release_start_link)
                release_start_soup = bs(request_release_start.content)
                all_date_release = release_start_soup.find_all("div","col-2 text")
                for j in range(len(all_date_release)-1,0,-1):
                    if all_date_release[j].text != "Unknown":
                        date = all_date_release[j].text 
                        break 
                    else:
                        date = "Unknown"
                    break
                    
    return date

In [10]:
def latest_scan(soup_series, series_id):
    if simulus_translate(soup_series, series_id) == True:
        date_scan = "Pass"
    else:
        if soup_series.find_all("div", "sContent")[5] == "" or soup_series.find_all("div", "sContent")[5].text.strip() == "N/A":
            date_scan = "Pass"
        else:
            release_link = "https://www.mangaupdates.com/releases.html?stype=series&search="+ series_id[0]
            request_release_link = myGet(release_link)
            release_link_soup = bs(request_release_link.content)
            release_date = release_link_soup.find_all("div","col-2 text")
            releaser_and_title = release_link_soup.find_all("div","col-4 text")
            releaser = []
            for i in range(len(releaser_and_title)):
                if i % 2 == 1: 
                    releaser.append(releaser_and_title[i].text)
            for j in range(len(release_date)): 
                if "Viz" not in releaser[j] and "MANGA Plus" not in releaser[j] and "Crunchyroll" not in releaser[j]:
                    date_scan = release_date[j].text
                    break
                break
    return date_scan


In [12]:
#Start out as scanlation or stimulus translation from the publisher
def publisher_first(soup_series, series_id):
    if soup_series.find_all("div","sContent")[5] == "" or soup_series.find_all("div","sContent")[5].text.strip() == "N/A":
        publisher_first = False
    else:
        release_link = "https://www.mangaupdates.com/releases.html?stype=series&search="+ series_id[0]
        request_release_link = myGet(release_link)
        release_link_soup = bs(request_release_link.content)
        page_number = release_link_soup.find_all("span","d-none")
        if page_number == []:
            group_name_title = release_link_soup.find_all("div","col-4 text")
            group_name = []
            for i in range(len(group_name_title)):
                if i % 2 == 1:
                    group_name.append(group_name_title[i].text)
            #first release
            x = len(group_name) - 1
            publisher_first = "Viz" in group_name[x] or "MANGA Plus" in group_name[x] or "Crunchyroll" in group_name[x]  
        else:
            page_number_only = re.findall(r'[0-9]+',page_number[0].text)[0]
            page_number_only = int(page_number_only)    
            last_page = release_link + "&page=" + str(page_number_only)
            request_last_page = myGet(last_page)
            last_page_soup = bs(request_last_page.content)
            group_name_title = last_page_soup.find_all("div","col-4 text")
            group_name = []
            for i in range(len(group_name_title)):
                if i % 2 == 1:
                    group_name.append(group_name_title[i].text)
            #first release
            x = len(group_name) - 1
            publisher_first = "Viz" in group_name[x] or "MANGA Plus" in group_name[x] or "Crunchyroll" in group_name[x]  

    return publisher_first
    
 

In [13]:
#Is the series stimulus translated?
def simulus_translate(soup_series, series_id):
    if soup_series.find_all("div","sContent")[5] == "":
        stimulus_translate = False
    else:
        release_link = "https://www.mangaupdates.com/releases.html?stype=series&search="+ series_id[0]
        request_release_link = myGet(release_link)
        release_link_soup = bs(request_release_link.content)
        title_and_releaser = release_link_soup.find_all("div","col-4 text")
        releaser = []
        count = 0
        for i in range(len(title_and_releaser)): 
            if i % 2 == 1: 
                releaser.append(title_and_releaser[i].text)
        for j in releaser: 
            if "Viz" in j or "MANGA Plus" in j or "Crunchyroll" in j:
                count += 1
        if count > 0:
            stimulus_translate = True
        else:
            stimulus_translate = False
        
    return stimulus_translate
    


In [14]:
#Genres of the manga
def genres(soup_series):
    All_Genres = ['Action','Adult','Adventure','Comedy','Doujinshi','Drama','Ecchi','Fantasy','Gender Bender','Harem','Hentai','Historical','Horror','Josei','Lolicon','Martial Arts','Mature','Mecha','Mystery','Psychological','Romance','School Life','Sci-fi','Seinen','Shotacon','Shoujo','Shoujo Ai','Shounen', 'Shounen Ai','Slice of Life','Smut','Sports','Supernatural','Tragedy','Yaoi','Yuri']
    all_sContent = soup_series.find_all("div","sContent")
    genres_series = all_sContent[14]
    genres_series_only = genres_series.find_all("u")
    genres_series_list = []
    genres_series_dict = {}
    for i in range(len(genres_series_only)-1): 
        genres_series_list.append(genres_series_only[i].text)
    for x in All_Genres: 
        if x in genres_series_list: 
            genres_series_dict[x] = 1
        else: 
            genres_series_dict[x] = 0
    return genres_series_dict
    


In [15]:
#Series' publish status: ongoing? complete? or unknown?
def publish_status(soup_series):
    publish_status_raw = soup_series.find_all("div","sContent")[6]
    publish_status_text = publish_status_raw.text.strip()
    if "Ongoing" in publish_status_text:
        status = "Ongoing"
    else: 
        if "Complete" in publish_status_text:
            status = "Complete"
        else:
            status = "Unknown"
    return status

In [16]:
def oneshot_or_not(soup_series):
    status_raw = soup_series.find_all("div","sContent")[6]
    return "Oneshot" in status_raw.text.strip()

In [17]:
#Series' scanlation status
def scanlate_status(soup_series):
    scanlated_status_raw = soup_series.find_all("div","sContent")[7]
    scanlated_status = scanlated_status_raw.text.strip()
    return scanlated_status

In [18]:
#Artist
def artist_name(soup_series):
    artist_raw = soup_series.find_all("div","sContent")[19]
    artist_only_raw = artist_raw.find_all("u")
    artist_list = [i.text for i in artist_only_raw]
    artist = ", ".join(artist_list)
    return artist


In [19]:
#Year publish
def year_publish(soup_series):
    year_original_publish = soup_series.find_all("div","sContent")[20].text.strip()
    return year_original_publish

In [425]:
#original publisher
# def original_publisher(soup_series):
#     publisher_name_raw = soup_series.find_all("div","sContent")[21]
#     publisher_name_list_raw = publisher_name_raw.find_all("u")
#     publisher_name_list = [i.text for i in publisher_name_list_raw]
#     publisher_name = ", ".join(publisher_name_list)
#     return publisher_name

In [20]:
#original publisher
def original_publisher(soup_series):
    publisher_name_raw = soup_series.find_all("div","sContent")[21]
    publisher_name_list_raw = publisher_name_raw.find_all("u")
    publisher_name_list = [i.text for i in publisher_name_list_raw]
    if "Add" in publisher_name_list:
        for i in publisher_name_list:
            if i == "Add":
                publisher_name_list.remove(i)
        publisher_name_left = re.findall(r'publisher=(.*?)"',str(publisher_name_raw))
        publisher_name_left = [i.replace('+', ' ') for i in publisher_name_left]
        publisher_name_list = publisher_name_list + publisher_name_left
        publisher_name = ", ".join(publisher_name_list)
    else:
        publisher_name = ", ".join(publisher_name_list)
    return publisher_name

In [21]:
#licensed in english
def license_or_not(soup_series):
    license_status = soup_series.find_all("div","sContent")[23].text.strip()
    return license_status

In [22]:
#Name of english publisher
def name_english_publisher(soup_series):
    name_raw = soup_series.find_all("div","sContent")[24]
    name_list_raw = name_raw.find_all("u")
    name_list = [i.text for i in name_list_raw]
    name = ", ".join(name_list)
    return name

In [23]:
#english publishing status ongoing? drop? hiatus?
def status_english_publishing(soup_series):
    publisher_and_status = soup_series.find_all("div","sContent")[24].text.strip()
    status_list = re.findall(r'\((.*?)\)',publisher_and_status)
    #check if the status was repeated
    status_list_norepeat = []
    for i in status_list: 
        if i not in status_list_norepeat:
            status_list_norepeat.append(i)
    status = ", ".join(status_list_norepeat)
    return status
#When searching for status, using df[df[publihsher].str.contains("complete")] 
#Regex has to include everything in the round brackets 
#because of the difference in the format during 20 years

In [24]:
def anime_or_not(soup_series):
    anime = soup_series.find_all("div", "sContent")[8].text.strip()
    if anime == "N/A" or anime == "": 
        return False
    else:
        return True
    

In [25]:
def rating(soup_series):
    series_rating_raw = soup_series.find_all("div","sContent")[11].text.strip()
    if series_rating_raw != "N/A":
        series_rating = re.findall(r'Average: (.*?) /', series_rating_raw)[0]
    else: 
        series_rating = "Not rated"
    return series_rating

In [26]:
def number_of_vote(soup_series):
    series_rating_raw = soup_series.find_all("div","sContent")[11].text.strip()
    if series_rating_raw != "N/A":
        vote_number = re.findall(r'\((.*?) votes\)', series_rating_raw)[0]
    else: 
        vote_number = "Not rated"
    return vote_number

In [89]:
# list_error_id = []
def series_cell(series_link):
    request_series_link = myGet(series_link)
    soup_series = bs(request_series_link.content.decode('utf-8', 'ignore'))
    series_id = re.findall(r'id=([0-9]+)',series_link)
    list_sContent = soup_series.find_all("div", "sContent")
    if len(list_sContent) < 27:
        list_error_id.append(series_id)
        print("sContent error at series_id = ", series_id)
        return None
    series_dict = {"ID": series_id[0], "Title": title(soup_series), "Type": type_comic(soup_series), "Number of Scanlation group": number_group_scanlate(soup_series), "Awarded": AwardorNot(series_id), "Japanese publihsers": original_publisher(soup_series), "Oneshot": oneshot_or_not(soup_series),"Scan start day": start_scan_date(soup_series, series_id), "Scan latest day": latest_scan(soup_series, series_id), "Digitally published first by publisher": publisher_first(soup_series, series_id), "Simulus translate": simulus_translate(soup_series, series_id), "Publish status":publish_status(soup_series), "Completely scanlated":scanlate_status(soup_series), "Artist": artist_name(soup_series), "Publish year": year_publish(soup_series), "Licensed": license_or_not(soup_series), "name of English publisher": name_english_publisher(soup_series), "English publishing status": status_english_publishing(soup_series), "Anime": anime_or_not(soup_series), "Rating": rating(soup_series), "Vote number": number_of_vote(soup_series)}  
    series_dict.update(genres(soup_series))
    series_cell = pd.DataFrame.from_dict(series_dict, orient="index").transpose()
    return series_cell

In [35]:
def series_dict(series_link):
    request_series_link = myGet(series_link)
    soup_series = bs(request_series_link.content)
    series_id = re.findall(r'id=([0-9]+)',series_link)
    series_dict = {"ID": series_id[0], "Title": title(soup_series), "Type": type_comic(soup_series), "Number of Scanlation group": number_group_scanlate(soup_series), "Awarded": AwardorNot(series_id), "Japanese publihsers": original_publisher(soup_series), "Oneshot": oneshot_or_not(soup_series),"Scan start day": start_scan_date(soup_series, series_id), "Scan latest day": latest_scan(soup_series, series_id), "Digitally published first by publisher": publisher_first(soup_series, series_id), "Simulus translate": simulus_translate(soup_series, series_id), "Publish status":publish_status(soup_series), "Completely scanlated":scanlate_status(soup_series), "Artist": artist_name(soup_series), "Publish year": year_publish(soup_series), "Licensed": license_or_not(soup_series), "name of English publisher": name_english_publisher(soup_series), "English publishing status": status_english_publishing(soup_series), "Anime": anime_or_not(soup_series), "Rating": rating(soup_series), "Vote number": number_of_vote(soup_series)}  
    return series_dict

TEST SOME TITLES

In [395]:
paradisekiss_link = "https://www.mangaupdates.com/series.html?id=1144"
paradisekiss_request = myGet(paradisekiss_link)
paradisekiss_soup = bs(paradisekiss_request.content)
original_publisher(paradisekiss_soup)

'Shodensha, Shueisha'

In [396]:
series_dict(paradisekiss_link)

{'ID': '1144',
 'Title': 'Paradise Kiss',
 'Type': 'Manga',
 'Number of Scanlation group': 1,
 'Awarded': False,
 'Japanese publihsers': 'Shodensha, Shueisha',
 'Oneshot': False,
 'Scan start day': '07/27/05',
 'Scan latest day': '07/13/06',
 'Digitally published first by publisher': False,
 'Simulus translate': False,
 'Publish status': 'Complete',
 'Completely scanlated': 'Yes',
 'Artist': 'YAZAWA Ai',
 'Publish year': '1999',
 'Licensed': 'Yes',
 'name of English publisher': 'TokyoPop, Vertical',
 'English publishing status': 'Expired / 5 Vols - Complete, 3 Vols - Complete',
 'Anime': True,
 'Rating': '8.5',
 'Vote number': '1057'}

In [397]:
%%time
link_japan_17 = "https://www.mangaupdates.com/series.html?id=7937"
id_japan17 = re.findall(r'id=([0-9]+)', link_japan_17)
series_dict(link_japan_17)
# request_japan17 = requests.get(link_japan_17)
# soup_japan17 = bs(request_japan17.content)
# soup_japan17.find_all("div","sContent")[5]

Wall time: 18.2 s


{'ID': '7937',
 'Title': 'Japan as Viewed by 17 Creators',
 'Type': 'Manga',
 'Number of Scanlation group': 0,
 'Awarded': False,
 'Japanese publihsers': 'Asuka Shinsha',
 'Oneshot': False,
 'Scan start day': 'Pass',
 'Scan latest day': 'Pass',
 'Digitally published first by publisher': False,
 'Simulus translate': False,
 'Publish status': 'Complete',
 'Completely scanlated': 'Yes',
 'Artist': 'ANNO Moyoco, Anthology, Aurélia Aurita, Benoît Peeters, David Prudhomme, Emmanuel Guibert, Étienne Davodeau, Fabrice Neaud, François Schuiten, Frédéric Boilet, HANAWA Kazuichi, IGARASHI Daisuke, Joann Sfar, Little Fish, MATSUMOTO Taiyou, Nicolas de Crécy, TAKAHAMA Kan, TANIGUCHI Jiro',
 'Publish year': '2005',
 'Licensed': 'Yes',
 'name of English publisher': 'Ponent Mon',
 'English publishing status': '1 Volume - Complete',
 'Anime': False,
 'Rating': '6.8',
 'Vote number': '47'}

In [ ]:
%%time
link_cyborg = "https://www.mangaupdates.com/series.html?id=26470"
series_dict(link_cyborg)

In [ ]:
link_love = "https://www.mangaupdates.com/series.html?id=5950"
series_dict(link_love)

In [100]:
link_cyborg = "https://www.mangaupdates.com/series.html?id=26470"
request_cyborg = requests.get(link_cyborg)
soup_cyborg = bs(request_cyborg.content)
id_cyborg = re.findall(r'id=([0-9]+)',link_cyborg)
latest_scan(soup_cyborg, id_cyborg)

'Pass'

In [ ]:
link_wanderingson = "https://www.mangaupdates.com/series.html?id=4802"
series_dict(link_wanderingson)

In [50]:
cyborg_publisher = soup_cyborg.find_all("div","sContent")[21]
cyborg_publisher

<div class="sContent"><a href="https://www.mangaupdates.com/publishers.html?id=9" title="Publisher Info"><u>Akita Shoten</u></a> (1966; 1989; 1994)
<br/>Fukkan.com [<a href="https://www.mangaupdates.com/submit.html?act=add_publisher&amp;publisher=Fukkan.com"><u>Add</u></a>] (2012)
<br/><a href="https://www.mangaupdates.com/publishers.html?id=100" title="Publisher Info"><u>Kodansha</u></a> (2014)
<br/><a href="https://www.mangaupdates.com/publishers.html?id=112" title="Publisher Info"><u>Media Factory</u></a> (1998; 2001)
<br/><a href="https://www.mangaupdates.com/publishers.html?id=149" title="Publisher Info"><u>Shogakukan</u></a> (1979; 2011)
<br/>
</div>

In [101]:
link_housenka = "https://www.mangaupdates.com/series.html?id=149531"
# request_housenka = requests.get(link_housenka)
# soup_housenka = bs(request_housenka.content)
# housenka_publisher = soup_housenka.find_all("div","sContent")[21]
# housenka_publisher
series_dict(link_housenka)

{'ID': '149531',
 'Title': 'Housenka',
 'Type': 'Manga',
 'Number of Scanlation group': 0,
 'Awarded': False,
 'Japanese publihsers': 'Bronze Sha',
 'Oneshot': False,
 'Scan start day': 'Pass',
 'Scan latest day': 'Pass',
 'Digitally published first by publisher': False,
 'Simulus translate': False,
 'Publish status': 'Complete',
 'Completely scanlated': 'No',
 'Artist': 'YAMADA Murasaki',
 'Publish year': '1980',
 'Licensed': 'No',
 'name of English publisher': '',
 'English publishing status': '',
 'Anime': False,
 'Rating': 'Not rated',
 'Vote number': 'Not rated'}

In [102]:
link_usubaka = "https://www.mangaupdates.com/series.html?id=31706"
series_dict(link_usubaka)
# request_usubaka = requests.get(link_usubaka)
# soup_usubaka = bs(request_usubaka.content)
# rating = soup_usubaka.find_all("div", "sContent")[11].text.strip()
# rating

{'ID': '31706',
 'Title': 'Usubaka Shouwa Blues',
 'Type': 'Manga',
 'Number of Scanlation group': 0,
 'Awarded': False,
 'Japanese publihsers': 'SeirinKogeisha',
 'Oneshot': False,
 'Scan start day': 'Pass',
 'Scan latest day': 'Pass',
 'Digitally published first by publisher': False,
 'Simulus translate': False,
 'Publish status': 'Complete',
 'Completely scanlated': 'No',
 'Artist': 'TOUYOU Kataoka',
 'Publish year': '2008',
 'Licensed': 'No',
 'name of English publisher': '',
 'English publishing status': '',
 'Anime': False,
 'Rating': 'Not rated',
 'Vote number': 'Not rated'}

In [ ]:
series_dict(series_link)

In [103]:
link_spyxfamily = "https://www.mangaupdates.com/series.html?id=153402"
series_dict(link_spyxfamily)

{'ID': '153402',
 'Title': 'Spy X Family',
 'Type': 'Manga',
 'Number of Scanlation group': 4,
 'Awarded': True,
 'Japanese publihsers': 'Shueisha',
 'Oneshot': False,
 'Scan start day': '09/08/19',
 'Scan latest day': 'Pass',
 'Digitally published first by publisher': True,
 'Simulus translate': True,
 'Publish status': 'Ongoing',
 'Completely scanlated': 'No',
 'Artist': 'ENDOU Tatsuya (遠藤達哉)',
 'Publish year': '2019',
 'Licensed': 'Yes',
 'name of English publisher': 'MANGA Plus, Viz',
 'English publishing status': '1 Volume - Ongoing',
 'Anime': False,
 'Rating': '9.1',
 'Vote number': '334'}

In [ ]:
link_dragonball = "https://www.mangaupdates.com/series.html?id=3106"
series_dict(link_dragonball)

In [398]:
link_hananoi = "https://www.mangaupdates.com/series.html?id=146249"
request_hananoi = myGet(link_hananoi)
soup_hananoi = bs(request_hananoi.content)
id_hananoi = re.findall(r'id=([0-9]+)', link_hananoi)
series_dict(link_hananoi)

{'ID': '146249',
 'Title': 'Hananoi-kun to Koi no Yamai',
 'Type': 'Manga',
 'Number of Scanlation group': 1,
 'Awarded': True,
 'Japanese publihsers': 'Kodansha',
 'Oneshot': False,
 'Scan start day': '01/06/18',
 'Scan latest day': '02/14/20',
 'Digitally published first by publisher': False,
 'Simulus translate': False,
 'Publish status': 'Ongoing',
 'Completely scanlated': 'No',
 'Artist': 'MORINO Megumi',
 'Publish year': '2017',
 'Licensed': 'Yes',
 'name of English publisher': 'Kodansha USA',
 'English publishing status': '5 Vols - Ongoing; digital',
 'Anime': False,
 'Rating': '8.4',
 'Vote number': '76'}

In [ ]:
soup_hananoi.find_all("div", "sContent")[8]

SCRAPE LINK FROM THE PAGE

In [119]:
link_1stpage_A = "https://www.mangaupdates.com/series.html?letter=A"
request_1stpage_A = requests.get(link_1stpage_A)
soup_1stpage_A = bs(request_1stpage_A.content)
series = soup_1stpage_A.find_all("a", "h-100 w-100")
series_link = [i.get('href') for i in series]
series_link

['https://www.mangaupdates.com/series.html?id=156195',
 'https://www.mangaupdates.com/series.html?id=153508',
 'https://www.mangaupdates.com/series.html?id=24345',
 'https://www.mangaupdates.com/series.html?id=8236',
 'https://www.mangaupdates.com/series.html?id=166153',
 'https://www.mangaupdates.com/series.html?id=89151',
 'https://www.mangaupdates.com/series.html?id=71469',
 'https://www.mangaupdates.com/series.html?id=110505',
 'https://www.mangaupdates.com/series.html?id=148820',
 'https://www.mangaupdates.com/series.html?id=54112',
 'https://www.mangaupdates.com/series.html?id=118082',
 'https://www.mangaupdates.com/series.html?id=110772',
 'https://www.mangaupdates.com/series.html?id=46102',
 'https://www.mangaupdates.com/series.html?id=149012',
 'https://www.mangaupdates.com/series.html?id=151170',
 'https://www.mangaupdates.com/series.html?id=131622',
 'https://www.mangaupdates.com/series.html?id=156624',
 'https://www.mangaupdates.com/series.html?id=102031',
 'https://www.man

In [129]:
%%time
df_series_test = pd.DataFrame() 
for i in series_link:
    df_series_test = df_series_test.append(series_cell(i))
df_series_test

Wall time: 1min 29s


,ID,Title,Type,Number of Scanlation group,Awarded,Japanese publihsers,Oneshot,Scan start day,Scan latest day,Digitally published first by publisher,...,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Smut,Sports,Supernatural,Tragedy,Yaoi,Yuri
0,156195,A ranku bouken-sha no surouraifu (Novel),Novel,1,False,Frontier Works,False,Unknown,Unknown,False,...,0,0,0,1,0,0,0,0,0,0
0,153508,A-Rank Boukensha No Slow Life,Manga,2,False,Futabasha,False,04/03/19,06/27/20,False,...,0,0,0,0,0,0,0,0,0,0
0,24345,Accidents,Manga,0,False,Shogakukan,False,Pass,Pass,False,...,0,1,0,0,0,0,0,0,0,0
0,8236,Adam to Eve no Houteishiki,Manga,0,False,Shogakukan,False,Pass,Pass,False,...,0,0,0,1,0,0,0,0,0,0
0,166153,Afterlife Class,Manhwa,1,False,Naver,False,04/26/20,04/26/20,False,...,0,0,0,1,0,0,0,0,0,0
0,89151,Ai no Jidai,Manga,0,False,"Kodansha, Satonaka Production",False,Pass,Pass,False,...,0,0,0,0,0,0,0,0,0,0
0,71469,Ai no Kuni Mogan Airu,Manga,0,False,Harlequin,False,Pass,Pass,False,...,0,0,0,0,0,0,0,0,0,0
0,110505,Akechi Keibu no Jikenbo,Manga,0,False,Kodansha,False,Pass,Pass,False,...,0,1,0,0,0,0,0,0,0,0
0,148820,Albino & Witch,Manhua,2,False,,False,03/02/18,04/01/18,False,...,0,0,0,1,0,0,1,0,0,1
0,54112,Amalfi no Hanayome,Manga,0,False,Harlequinsha,False,Pass,Pass,False,...,0,0,0,0,0,0,0,0,0,0


In [90]:
def each_title_page(title_page):
    df_title = pd.DataFrame()
    req_title_page = myGet(title_page)
    soup_title_page = bs(req_title_page.content.decode('utf-8', 'ignore'))
    link_title_collection = soup_title_page.find_all("a", "h-100 w-100")
    list_titles = [i.get("href") for i in link_title_collection]
    for i in list_titles:
        df_title = df_title.append(series_cell(i))
    return df_title

In [468]:
df_test_title = pd.DataFrame()
page_13 = 'https://www.mangaupdates.com/series.html?page=13&licensed=yes'
req_title_page = myGet(page_13)
soup_title_page = bs(req_title_page.content)
link_title_collection = soup_title_page.find_all("a", "h-100 w-100")
list_titles = [i.get("href") for i in link_title_collection]

title_13_req = myGet(list_titles[7])
soup_title_13 = bs(title_13_req.content)
soup_title_13.find_all("div", "sContent")
# df_test_title = df_test_title.append(series_cell(list_titles[3]))

[<div class="sContent" style="text-align:justify">
 <script type="text/javascript"><!--
 function dispdescMore()
 {
 	document.getElementById('div_desc_link').style.display='none';
 	document.getElementById('div_desc_more').style.display=''
 }
 function dispdescLess()
 {
 	document.getElementById('div_desc_link').style.display='';
 	document.getElementById('div_desc_more').style.display='none'
 }
 //--></script>
 <div id="div_desc_link">
 from fictiondb:<br/>Trevennon had a dark and tragic history. <br/><br/>As a child, Morwenna had listened to her mother's stories of Trevennon, her old home. Morwenna had pictured a castle full of the magic of love, standing high on the cliffs of Cornwall. <br/><br/>So when tragedy struck the eighteen-year-old Morwenna, she fled to Trevennon. Contrary to her expectations, she found a house full of unhappiness and hostility - Dominic Trevennon's hostility. <br/><br/>But strangely, Dominic capture Morwenna's heart as completely as her mother's stories ha

In [40]:
list_error_id = [['157629'], ['142630']]

In [92]:
import datetime
start_scrape_title = datetime.datetime.now()
for i in [78, 98, 104, 114, 130, 135, 187]:
    title_page = 'https://www.mangaupdates.com/series.html?page={}&licensed=yes'.format(i)
    each_title_page(title_page).to_csv('title_license_{}'.format(i))
    time_process = datetime.datetime.now() - start_scrape_title
    print(time_process)

0:05:01.249026
0:10:28.027890
0:16:02.306975
0:21:11.627406
0:25:56.979021
0:32:13.241446
0:37:40.202481


In [39]:
list_error_id

NameError: name 'list_error_id' is not defined

In [497]:
df_title = pd.DataFrame()
title_page = 'https://www.mangaupdates.com/series.html?page=297&licensed=yes'
req_title_page = myGet(title_page)
soup_title_page = bs(req_title_page.content)
link_title_collection = soup_title_page.find_all("a", "h-100 w-100")
list_titles = [i.get("href") for i in link_title_collection]
# for i in list_titles:
#     df_title = df_title.append(series_cell(i))
# df_title


In [523]:

    error_title_req = myGet(i)
    soup_error_title = bs(error_title_req.content)
    error_id = re.findall(r'id=([0-9]+)',i)
    try:
        latest_scan(soup_error_title, error_id)
    except:
        print(error_id)

# SCRAPE THE LEFTOVER TITLES


In [93]:
list_error_id = [['157629'],
 ['142630'],
 ['7684'],
 ['10562'],
 ['142661'],
 ['56123'],
 ['21729'],
 ['110961'],
 ['49842'],
 ['99938'],
 ['170300'],
['35759']]

In [97]:
list_error_id_short = [i[0] for i in list_error_id]

In [98]:
df_title_leftover = pd.DataFrame()

In [99]:
for i in list_error_id_short:
    link_title = "https://www.mangaupdates.com/series.html?id={}".format(i)
    df_title_leftover = df_title_leftover.append(series_cell(link_title))

In [101]:
df_title_leftover.to_csv('title_license_401')